# OVERVIEW

In this notebook, we will check if there is any predictive power in the meta-features that can be extracted from the DCM images. We will write a function to extract a few meta-features and pixel-based features taking advantage of the parallelization and build a cross-validation based LightGBM pipeline to predict one image-level and nine exam-level labels.

# PREPARATIONS

In [ ]:
##### PACKAGES

import os
import numpy as np
import pandas as pd

import pydicom as dcm
from tqdm import tqdm

import cv2
import vtk
from vtk.util import numpy_support

from joblib import Parallel, delayed
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

import seaborn as sns
sns.set(style = 'dark')
import matplotlib.pyplot as plt

# EXTRACT META-FEATURES

Each DCM image contains meta-data in addition to the pixel arrays. We can look at the meta-data using the following code as an example:

In [ ]:
##### EXAMPLE META-DATA

image = dcm.dcmread('/kaggle/input/rsna-str-pulmonary-embolism-detection/test/e0b02c539fb7/9ca8bffc6624/a95a398ceaf9.dcm')
image

Let's write a function to extract meta-features given the image path. We will also use windowing to extract stats on the pixel values from differen image windows.

In [ ]:
##### META-FEATURES EXTRACTOR

def window(img, WL = 50, WW = 350):
    upper, lower = WL + WW//2, WL - WW//2
    X = np.clip(img.copy(), lower, upper)
    X = X - np.min(X)
    X = X / np.max(X)
    return X

def extract_meta_feats(img):

    img_id = img.split('/')[-1].replace('.dcm', '')
    image  = dcm.dcmread(img)
    
    
    ### META-FEATURES
    
    pixelspacing      = image.PixelSpacing[0]
    slice_thicknesses = image.SliceThickness
    kvp               = image.KVP
    table_height      = image.TableHeight
    x_ray             = image.XRayTubeCurrent
    exposure          = image.Exposure
    modality          = image.Modality
    rot_direction     = image.RotationDirection 
    instance_number   = image.InstanceNumber
    
    
    ### PIXEL-BASED FEATURES
    
    reader = vtk.vtkDICOMImageReader()
    reader.SetFileName(img)
    reader.Update()
    _extent = reader.GetDataExtent()
    ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

    ConstPixelSpacing = reader.GetPixelSpacing()
    imageData  = reader.GetOutput()
    pointData  = imageData.GetPointData()
    arrayData  = pointData.GetArray(0)
    ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
    ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order = 'F')
    ArrayDicom = cv2.resize(ArrayDicom, (512,512))

    img = ArrayDicom.astype(np.int16)
    img[img <= -1000] = 0

    intercept = reader.GetRescaleOffset()
    slope     = reader.GetRescaleSlope()
    if slope != 1:
        img = slope * img.astype(np.float64)
        img = img.astype(np.int16)
    img += np.int16(intercept)

    hu_min  = np.min(img)
    hu_mean = np.mean(img)
    hu_max  = np.max(img)
    hu_std =  np.std(img)
    
    
    ### WINDOW-BASED FEATURES
    
    img_lung = window(img, WL = -600, WW = 1500)
    img_medi = window(img, WL = 40,   WW = 400)
    img_pesp = window(img, WL = 100,  WW = 700)
    
    lung_mean = np.mean(img_lung)
    lung_std  = np.std(img_lung)
    
    medi_mean = np.mean(img_medi)
    medi_std = np.std(img_medi)
    
    pesp_mean = np.mean(img_pesp)
    pesp_std  = np.std(img_pesp)
    
    
    return [img_id, 
            pixelspacing, slice_thicknesses, kvp, table_height, x_ray, exposure, modality, rot_direction, instance_number,
            hu_min, hu_mean, hu_max, hu_std,
            lung_mean, lung_std, medi_mean, medi_std, pesp_mean, pesp_std,
           ]

Since the data set is very large (moree than 1.7 million images), we need to parallelize feature extraction. Below, we construct a list of all test images and extract their meta-features using `Parallel`. To speed up the process, we extract meta-features from training images [in a separate notebook](https://www.kaggle.com/kozodoi/extract-meta-features-from-training-images) using the same function and import them here.

In [ ]:
##### IMAGE PATH FOR TEST IMAGES

im_path = []
test_path = '../input/rsna-str-pulmonary-embolism-detection/test/'
for i in tqdm(os.listdir(test_path)): 
    for j in os.listdir(test_path + i):
        for k in os.listdir(test_path + i + '/' + j):
            x = test_path + i + '/' + j + '/' + k
            im_path.append(x)

In [ ]:
##### EXTRACT META-FEATURES

results = Parallel(n_jobs = -1, verbose = 1)(map(delayed(extract_meta_feats), im_path))

In [ ]:
##### CONSTRUCT TEST DATAFRAME

test_meta = pd.DataFrame(results, columns = ['SOPInstanceUID', 
                                             'pixelspacing',
                                             'slice_thicknesses',
                                             'kvp',
                                             'table_height',
                                             'x_ray_tube_current',
                                             'exposure',
                                             'modality',
                                             'rotation_direction',
                                             'instance_number',
                                             'hu_min',
                                             'hu_mean',
                                             'hu_max',
                                             'hu_std',
                                             'lung_mean',
                                             'lung_std',
                                             'medi_mean',
                                             'medi_std',
                                             'pesp_mean',
                                             'pesp_std'])
test = pd.read_csv('/kaggle/input/rsna-str-pulmonary-embolism-detection/test.csv')
test = test.merge(test_meta, how = 'left', on = 'SOPInstanceUID')
test.head()

In [ ]:
##### IMPORT SAVED TRAIN DATA FEATURES

train      = pd.read_csv('/kaggle/input/rsna-str-pulmonary-embolism-detection/train.csv')
train_meta = pd.read_csv('/kaggle/input/extract-meta-features-from-training-images/train_meta.csv')
train      = train.merge(train_meta, how = 'left', on = 'SOPInstanceUID')
train.head()

In [ ]:
##### ADDITIONAL FEATURES

train['n_images_per_study'] = train['StudyInstanceUID'].map(train.groupby(['StudyInstanceUID']).SOPInstanceUID.nunique())
test['n_images_per_study']  = test['StudyInstanceUID'].map(test.groupby(['StudyInstanceUID']).SOPInstanceUID.nunique())

train['relative_instance_number'] = train['instance_number'] / train['n_images_per_study']
test['relative_instance_number']  = test['instance_number'] / test['n_images_per_study']

# MODELING: IMAGE-LEVEL LABELS

We can now start modeling! First, let's tackle the image-level label: `pe_present_on_image`. We will create `X` and `y` data sets and set up a 5-fold cross-validation. We are using a LightGBM classifier with a binary log-loss objective function.

In [ ]:
##### SEPARATE X AND Y

features = ['pixelspacing', 'slice_thicknesses', 'kvp', 'table_height', 'x_ray_tube_current', 'exposure', 
            'n_images_per_study', 'instance_number', 'relative_instance_number',
            'hu_min', 'hu_mean', 'hu_max', 'hu_std',
            'lung_mean', 'lung_std', 'medi_mean', 'medi_std', 'pesp_mean', 'pesp_std']
label    = 'pe_present_on_image'

X = train[features + ['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID']]
y = train[label]
print(X.shape, y.shape)

test[label] = np.nan

In [ ]:
##### MODELING PARAMS

# random seed
seed = 23

# rounds and options
stop_rounds = 100
verbose     = 200

# LGB parameters
lgb_params = {
    'objective':        'binary',
    'metrics':          'logloss',
    'n_estimators':     10000,
    'learning_rate':    0.01,
    'bagging_fraction': 0.8,
    'feature_fraction': 0.8,
    'lambda_l1':        0.1,
    'lambda_l2':        0.1,
    'scale_pos_weiht':  1,
    'silent':           True,
    'verbosity':        -1,
    'nthread' :         -1,
    'random_state':     seed,
}

# partitioning
'''
Adapted from https://www.kaggle.com/khyeh0719/stratified-validation-strategy
'''
folds  = 10
xfolds = pd.read_csv('/kaggle/input/stratified-validation-strategy/rsna_train_splits_fold_20.csv')
train  = train.merge(xfolds[['StudyInstanceUID', 'fold']], how = 'left', on = 'StudyInstanceUID')
train.loc[train.fold >= folds, 'fold'] = train.loc[train.fold >= folds, 'fold'] - 10
train.fold.value_counts()

In [ ]:
##### MODELING

# placeholders
oof         = np.zeros(len(X))
test_preds  = np.zeros(len(test))
importances = pd.DataFrame()

# cross-validation
for fold in range(folds):
    
    # display info
    print('-' * 30)
    print('FOLD {:d}/{:d}'.format(fold + 1, folds))    
    print('-' * 30)
    
    # fold idx
    trn_idx = train.loc[train.fold != fold].index
    val_idx = train.loc[train.fold == fold].index
     
    # extract samples
    X_train, y_train = X.iloc[trn_idx][features], y.iloc[trn_idx]
    X_valid, y_valid = X.iloc[val_idx][features], y.iloc[val_idx]
    
    # modeling
    clf = lgb.LGBMClassifier(**lgb_params) 
    clf = clf.fit(X_train, y_train, 
                  eval_set              = [(X_train, y_train), (X_valid, y_valid)],
                  eval_metric           = 'logloss',
                  early_stopping_rounds = stop_rounds,
                  verbose               = verbose)
    
    # prediction
    oof[val_idx] = clf.predict_proba(X_valid)[:,1]
    test_preds  += clf.predict_proba(test[features])[:,1] / folds
    
    # feature importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df['Feature']    = features
    fold_importance_df['Importance'] = clf.feature_importances_
    fold_importance_df['Fold']       = fold + 1
    importances = pd.concat([importances, fold_importance_df], axis = 0)
    print('-' * 30)
    print('')

# print performance
print('-' * 30)
print('OOF LOG-LOSS: {:.6f}'.format(log_loss(y, oof)))
print('-' * 30)

# store test preds
test[label] = test_preds

In [ ]:
##### FEATURE IMPORTANCE

cols = importances[['Feature', 'Importance']].groupby('Feature').mean().sort_values(by = 'Importance', ascending = False).index
importance = importances.loc[importances.Feature.isin(cols)]

fig = plt.figure(figsize = (8, 5))
sns.barplot(x = 'Importance', y = 'Feature', data = importance.sort_values(by = 'Importance', ascending = False), ci = 0)
plt.tight_layout()
plt.savefig('varimp_' + label + '.pdf')

# MODELING: EXAM-LEVEL LABELS

Now, we also have nine exam-level labels. Let's perform modeling for each of the labels separately. It is important to aggregate the data by exam first to perform modeling on the exam level. We will use a similar pipeline as with the image-level label. We can also add OOF image-level predictions of `pe_present_on_image` as a feature.

In [ ]:
##### ADD PE PREDICTION AS FEATURE

train['pred_pe_present_on_image'] = oof
test['pred_pe_present_on_image']  = test_preds

In [ ]:
##### FEATURES AND LABELS

features = ['pixelspacing', 'slice_thicknesses', 'kvp', 'table_height', 
            'x_ray_tube_current', 'exposure', 'n_images_per_study',
            'pred_pe_present_on_image', 'hu_min', 'hu_mean', 'hu_max', 'hu_std',
            'lung_mean', 'lung_std', 'medi_mean', 'medi_std', 'pesp_mean', 'pesp_std']

labels   = ['negative_exam_for_pe',
            'rv_lv_ratio_gte_1',
            'rv_lv_ratio_lt_1',
            'leftsided_pe',
            'chronic_pe',
            'rightsided_pe',
            'acute_and_chronic_pe',
            'central_pe',
            'indeterminate']

In [ ]:
##### MODELING PARAMS

# random seed
seed = 23

# rounds and options
stop_rounds = 200
verbose     = False

# LGB parameters
lgb_params = {
    'objective':        'binary',
    'metrics':          'logloss',
    'n_estimators':     10000,
    'learning_rate':    0.01,
    'bagging_fraction': 0.8,
    'feature_fraction': 0.8,
    'lambda_l1':        0.1,
    'lambda_l2':        0.1,
    'scale_pos_weiht':  1,
    'silent':           True,
    'verbosity':        -1,
    'nthread' :         -1,
    'random_state':     seed,
}

# partitioning
folds = 10
skf   = StratifiedKFold(n_splits     = folds, 
                        shuffle      = True, 
                        random_state = seed)

In [ ]:
##### MODELING

# placeholders
oof = np.zeros((train['StudyInstanceUID'].nunique(), len(labels)))

# modeling loop
for label in labels:
    
    # display info
    print('-' * 30)
    print('LABEL = {}'.format(label))    
    print('-' * 30)
    
    ### SEPARATE X AND Y AND TRANSFORM DATA 
    
    X_exam = train.groupby('StudyInstanceUID')[features].agg(['mean', 'min', 'max', 'std']).reset_index(drop = True)
    X_exam.columns = ['_'.join(col).strip() for col in X_exam.columns.values]
    X_exam = X_exam.sort_index()

    test_exam = test.groupby('StudyInstanceUID')[features].agg(['mean', 'min', 'max', 'std']).reset_index(drop = False)
    test_exam.columns = ['_'.join(col).strip() for col in test_exam.columns.values]
    test_exam.rename({'StudyInstanceUID_': 'StudyInstanceUID'}, axis = 1, inplace = True)    
    test_exam = test_exam.sort_index()

    y_exam = train.groupby('StudyInstanceUID')[label].agg('mean').reset_index(drop = True)

    # placeholders
    test_preds  = np.zeros(len(test_exam))
    importances = pd.DataFrame()

    # cross-validation
    for fold, (trn_idx, val_idx) in enumerate(skf.split(X_exam, y_exam)):

        # display info
        print('- FOLD {:d}/{:d}...'.format(fold + 1, folds))    

        # extract samples
        X_train, y_train = X_exam.iloc[trn_idx], y_exam.iloc[trn_idx]
        X_valid, y_valid = X_exam.iloc[val_idx], y_exam.iloc[val_idx]

        # modeling
        clf = lgb.LGBMClassifier(**lgb_params) 
        clf = clf.fit(X_train, y_train, 
                      eval_set              = [(X_train, y_train), (X_valid, y_valid)],
                      eval_metric           = 'logloss',
                      early_stopping_rounds = stop_rounds,
                      verbose               = verbose)

        # prediction
        oof[val_idx, labels.index(label)] = clf.predict_proba(X_valid)[:,1]
        test_preds += clf.predict_proba(test_exam[X_train.columns])[:,1] / folds
        
        # feature importance
        fold_importance_df = pd.DataFrame()
        fold_importance_df['Feature']    = X_train.columns
        fold_importance_df['Importance'] = clf.feature_importances_
        fold_importance_df['Fold']       = fold + 1
        importances = pd.concat([importances, fold_importance_df], axis = 0)

    # print performance
    print('- OOF LOG-LOSS: {:.6f}'.format(log_loss(y_exam, oof[:, labels.index(label)])))
    print('-' * 30)
    print('')
    
    # feature importance
    cols = importances[['Feature', 'Importance']].groupby('Feature').mean().sort_values(by = 'Importance', ascending = False).index
    importance = importances.loc[importances.Feature.isin(cols)]
    fig = plt.figure(figsize = (8, 5))
    sns.barplot(x = 'Importance', y = 'Feature', data = importance.sort_values(by = 'Importance', ascending = False), ci = 0)
    plt.savefig('varimp_' + label + '.pdf')
    
    # store test preds
    test_exam[label] = test_preds
    test = test.merge(test_exam[['StudyInstanceUID', label]])

# PERFORMANCE EVALUATION

Let's evaluate performance of our predictions in terms of the competition metric: weighted log-loss over ten labels.

In [ ]:
########## CONSTRUCT OOF DF

oof_df = pd.DataFrame(oof)
oof_df.columns = labels
oof_df.insert(0, 'StudyInstanceUID', train.groupby('StudyInstanceUID').agg('mean').reset_index(drop = False)['StudyInstanceUID'])
oof_df = oof_df.merge(train[['StudyInstanceUID' ,'SeriesInstanceUID', 'SOPInstanceUID', 'pred_pe_present_on_image']],
                      on  = 'StudyInstanceUID',
                      how = 'right')
oof_df = oof_df.rename(columns = {'pred_pe_present_on_image': 'pe_present_on_image'})
oof_df.head()

In [ ]:
########## COMPETITION METRIC

'''
Adapted from:
- https://www.kaggle.com/khyeh0719/0929-updated-rsna-competition-metric
- https://www.kaggle.com/kingstying/rsna-ped-check-metric
'''

def competition_score(df_probs, df):
    
    def cross_entropy(targets, predictions, epsilon = 1e-12):
        predictions = np.clip(predictions, epsilon, 1. - epsilon)
        ce = -(targets*np.log(predictions) + (1.-targets)*np.log(1.-predictions))
        return ce
    
    qi = df[['pe_present_on_image', 'StudyInstanceUID']].groupby('StudyInstanceUID').mean().reset_index()
    qi.columns = ['StudyInstanceUID', 'qi']
    
    cols_label = ['pe_present_on_image',
                  'negative_exam_for_pe', 'indeterminate', 'chronic_pe',
                  'acute_and_chronic_pe', 'central_pe', 'leftsided_pe',
                  'rightsided_pe', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1']
    
    weights = [0.07361963,
               0.0736196319, 0.09202453988, 0.1042944785,
               0.1042944785, 0.1877300613, 0.06257668712,
               0.06257668712, 0.2346625767, 0.0782208589]
    
    assert (df['SOPInstanceUID'] == df_probs['SOPInstanceUID']).all(), f'SOPInstanceUID not match!'
    
    target_exam = df[['StudyInstanceUID']       + cols_label[1:]].groupby('StudyInstanceUID').mean()
    probs_exam  = df_probs[['StudyInstanceUID'] + cols_label[1:]].groupby('StudyInstanceUID').mean()
    
    score_exam = []
    epsilon    = 1e-12
    
    for col,w in zip(cols_label[1:],weights[1:]):
        score = log_loss(target_exam[col].values, np.clip(probs_exam[col].values, epsilon, 1. - epsilon)) * w 
        score = score * target_exam.shape[0] 
        score_exam.append(score)
        
    score_exam = np.sum(score_exam)
    
    df_probs = pd.merge(df_probs, qi, on = 'StudyInstanceUID', how = 'inner')
    df_probs['target-pe_present_on_image'] = df['pe_present_on_image']
    

    df_probs['score_img'] = \
    df_probs[['target-pe_present_on_image', 'pe_present_on_image', 'qi']].apply(lambda x:cross_entropy(x[0],x[1])*x[2]*weights[0], axis=1)
    
    score_img = df_probs['score_img'].sum()
    
    total_score   = score_exam + score_img
    total_weights = np.sum(weights[1:])*df.StudyInstanceUID.nunique() + np.sum(weights[0]*df_probs['qi'].values)
    
    return total_score / total_weights

In [ ]:
########## CHECK PERFORMANCE

print('Competition score: {:.6f}'.format(competition_score(oof_df, train)))

# PREPARE SUBMISSION

We can now prepare a submission file with our predictions. To do that, we need to transform predictions to the required format. This is done in the code snippets below.

In [ ]:
##### CHECK PREDICTIONS

sub_df = test.copy()
sub_df.head()

In [ ]:
##### EXAM-LEVEL PREDS

exam_label_names = ['negative_exam_for_pe',
                    'rv_lv_ratio_gte_1',
                    'rv_lv_ratio_lt_1',
                    'leftsided_pe',
                    'chronic_pe',
                    'rightsided_pe',
                    'acute_and_chronic_pe',
                    'central_pe',
                    'indeterminate']

# list of test ids
test_exam_ids = []
for v in test.StudyInstanceUID:
    if v not in test_exam_ids:
        test_exam_ids.append(v)
        
# placeholders
ids    = []
labels = []

# aggregate exam-level preds
for label in tqdm(exam_label_names):
    for key in test_exam_ids:
        tmp_sub = sub_df.loc[sub_df.StudyInstanceUID == key]
        ids.append('_'.join([key, label]))
        labels.append(np.max(tmp_sub[label]))

In [ ]:
##### IMAGE-LEVEL PREDS

ids    += sub_df.SOPInstanceUID.tolist()
labels += sub_df['pe_present_on_image'].tolist()

In [ ]:
##### SUBMISSION FILE

sub = pd.DataFrame({'id':ids, 'label': labels})
sub.to_csv('submission.csv', index = False)
sub.shape

# CHECK LABEL CONSISTENCY

Finally, let's check label consistency using a function from [this notebook](https://www.kaggle.com/kozodoi/checking-the-label-consistency-requirements).

In [ ]:
def check_consistency(sub, test):
    
    '''
    Checks label consistency and returns the errors
    
    Args:
    sub   = submission dataframe (pandas)
    test  = test.csv dataframe (pandas)
    '''
    
    # EXAM LEVEL
    for i in test['StudyInstanceUID'].unique():
        df_tmp = sub.loc[sub.id.str.contains(i, regex = False)].reset_index(drop = True)
        df_tmp['StudyInstanceUID'] = df_tmp['id'].str.split('_').str[0]
        df_tmp['label_type']       = df_tmp['id'].str.split('_').str[1:].apply(lambda x: '_'.join(x))
        del df_tmp['id']
        if i == test['StudyInstanceUID'].unique()[0]:
            df = df_tmp.copy()
        else:
            df = pd.concat([df, df_tmp], axis = 0)
    df_exam = df.pivot(index = 'StudyInstanceUID', columns = 'label_type', values = 'label')
    
    # IMAGE LEVEL
    df_image = sub.loc[sub.id.isin(test.SOPInstanceUID)].reset_index(drop = True)
    df_image = df_image.merge(test, how = 'left', left_on = 'id', right_on = 'SOPInstanceUID')
    df_image.rename(columns = {"label": "pe_present_on_image"}, inplace = True)
    del df_image['id']
    
    # MERGER
    df = df_exam.merge(df_image, how = 'left', on = 'StudyInstanceUID')
    ids    = ['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID']
    labels = [c for c in df.columns if c not in ids]
    df = df[ids + labels]
    
    # SPLIT NEGATIVE AND POSITIVE EXAMS
    df['positive_images_in_exam'] = df['StudyInstanceUID'].map(df.groupby(['StudyInstanceUID']).pe_present_on_image.max())
    df_pos = df.loc[df.positive_images_in_exam >  0.5]
    df_neg = df.loc[df.positive_images_in_exam <= 0.5]
    
    # CHECKING CONSISTENCY OF POSITIVE EXAM LABELS
    rule1a = df_pos.loc[((df_pos.rv_lv_ratio_lt_1  >  0.5)  & 
                         (df_pos.rv_lv_ratio_gte_1 >  0.5)) | 
                        ((df_pos.rv_lv_ratio_lt_1  <= 0.5)  & 
                         (df_pos.rv_lv_ratio_gte_1 <= 0.5))].reset_index(drop = True)
    rule1a['broken_rule'] = '1a'
    rule1b = df_pos.loc[(df_pos.central_pe    <= 0.5) & 
                        (df_pos.rightsided_pe <= 0.5) & 
                        (df_pos.leftsided_pe  <= 0.5)].reset_index(drop = True)
    rule1b['broken_rule'] = '1b'
    rule1c = df_pos.loc[(df_pos.acute_and_chronic_pe > 0.5) & 
                        (df_pos.chronic_pe           > 0.5)].reset_index(drop = True)
    rule1c['broken_rule'] = '1c'
    rule1d = df_pos.loc[(df_pos.indeterminate        > 0.5) | 
                        (df_pos.negative_exam_for_pe > 0.5)].reset_index(drop = True)
    rule1d['broken_rule'] = '1d'

    # CHECKING CONSISTENCY OF NEGATIVE EXAM LABELS
    rule2a = df_neg.loc[((df_neg.indeterminate        >  0.5)  & 
                         (df_neg.negative_exam_for_pe >  0.5)) | 
                        ((df_neg.indeterminate        <= 0.5)  & 
                         (df_neg.negative_exam_for_pe <= 0.5))].reset_index(drop = True)
    rule2a['broken_rule'] = '2a'
    rule2b = df_neg.loc[(df_neg.rv_lv_ratio_lt_1     > 0.5) | 
                        (df_neg.rv_lv_ratio_gte_1    > 0.5) |
                        (df_neg.central_pe           > 0.5) | 
                        (df_neg.rightsided_pe        > 0.5) | 
                        (df_neg.leftsided_pe         > 0.5) |
                        (df_neg.acute_and_chronic_pe > 0.5) | 
                        (df_neg.chronic_pe           > 0.5)].reset_index(drop = True)
    rule2b['broken_rule'] = '2b'
    
    # MERGING INCONSISTENT PREDICTIONS
    errors = pd.concat([rule1a, rule1b, rule1c, rule1d, rule2a, rule2b], axis = 0)
    
    # OUTPUT
    print('Found', len(errors), 'inconsistent predictions')
    return errors

In [ ]:
##### CHECK

test_df = pd.read_csv('/kaggle/input/rsna-str-pulmonary-embolism-detection/test.csv')
errors  = check_consistency(sub, test_df)
errors.broken_rule.value_counts()

For now, we will ingore the label inconsistencies and submit the predictions as is. 

Judging by the submission score, there is not much signal in the meta-data, as a simple mean baseline ouperforms the above code. Still, it was useful to check if there is any correlation in there. I hope you will find this notebook useful!